# Install required dependencies

In [1]:
!pip install datasets evaluate transformers rouge-score nltk
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=dbfe8e19212feac3f2d78f616b4770b17e32017d859c404f48228585405cbd83
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [22]:
#Import huggingface
from huggingface_hub import notebook_login

notebook_login()

In [4]:
!apt install git-lfs

# Fine-tuning a model on a text-to-text task

In [5]:
#Call the model name
model_checkpoint = "t5-small"

## Custom Dataset

In [7]:
import pandas as pd
from datasets import Dataset, DatasetDict, Value
from evaluate import load

df = pd.read_json("/content/manimml_data.json")
train_df = df[:int((len(df)+1)*0.80)]
val_df = df[int((len(df)+1)*0.80):]

def create_dataset(df):
  dataset = Dataset.from_dict({"description": df['text'], "program":df['code']})
  dataset = dataset.cast_column("description", Value(dtype='string', id=None))
  dataset = dataset.cast_column("program", Value(dtype='string', id=None))
  return dataset

train_dataset = create_dataset(train_df)
val_dataset = create_dataset(val_df)

metric = load("rouge")


Casting the dataset:   0%|          | 0/11200 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/11200 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/2800 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/2800 [00:00<?, ? examples/s]

In [8]:
dataset = DatasetDict({
     "train": train_dataset,
     "validation": val_dataset,
     }
)

In [9]:
dataset["train"][0]

{'description': 'This code snippet defines a neural network with a single convolutional layer. The layer has 3 feature maps, each with a size of 5x5 pixels. The filters used in the convolutional operation have a size of 5x5 pixels as well, and are spaced 0.34 pixels apart. The activation function used in the layer is ReLU (Rectified Linear Unit). The layer spacing, which determines the distance between the feature maps, is set to 0.47. This configuration allows the network to extract features from images with a spatial resolution of 5x5 pixels, using 3 feature maps and a relatively dense set of filters. The ReLU activation function helps to introduce non-linearity in the network, while the layer spacing controls the amount of information that is preserved between feature maps.',
 'program': 'nn=NeuralNetwork(Convolutional2DLayer(num_feature_maps=3, feature_map_size=5, filter_size=5, filter_spacing=0.34, activation_function=ReLU), layer_spacing: 0.47)'}

## Preprocessing the data

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [11]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "DSL: "
else:
    prefix = ""

In [12]:
max_input_length = 1024
max_target_length = 1024

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["description"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["program"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [13]:
preprocess_function(dataset['train'][:2])

{'input_ids': [[309, 5629, 10, 100, 1081, 3, 20317, 4995, 17692, 3, 9, 24228, 1229, 28, 3, 9, 712, 975, 24817, 138, 3760, 5, 37, 3760, 65, 220, 1451, 8111, 6, 284, 28, 3, 9, 812, 13, 305, 226, 755, 15386, 5, 37, 10607, 261, 16, 8, 975, 24817, 138, 2986, 43, 3, 9, 812, 13, 305, 226, 755, 15386, 38, 168, 6, 11, 33, 628, 26, 4097, 3710, 15386, 3943, 5, 37, 5817, 257, 1681, 261, 16, 8, 3760, 19, 419, 9138, 41, 1649, 75, 17, 3676, 4919, 291, 5579, 137, 37, 3760, 30024, 6, 84, 2082, 7, 8, 2357, 344, 8, 1451, 8111, 6, 19, 356, 12, 4097, 4177, 5, 100, 5298, 1250, 8, 1229, 12, 5819, 753, 45, 1383, 28, 3, 9, 15208, 3161, 13, 305, 226, 755, 15386, 6, 338, 220, 1451, 8111, 11, 3, 9, 4352, 13809, 356, 13, 10607, 5, 37, 419, 9138, 5817, 257, 1681, 1691, 12, 4277, 529, 18, 747, 291, 485, 16, 8, 1229, 6, 298, 8, 3760, 30024, 7415, 8, 866, 13, 251, 24, 19, 18570, 344, 1451, 8111, 5, 1], [309, 5629, 10, 901, 3535, 6, 27, 31, 26, 36, 1095, 12, 199, 55, 264, 6, 34, 1416, 114, 25, 31, 162, 530, 3, 9, 24228

In [14]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/11200 [00:00<?, ? examples/s]

Map:   0%|          | 0/2800 [00:00<?, ? examples/s]

## Fine-tuning the model

In [15]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [24]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-manimml-1.1",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [18]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [25]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [26]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.810800,0.048984,26.763400,21.531300,26.758600,26.754000,19.000000
2,0.127900,0.030858,26.803700,21.595500,26.804800,26.795900,19.000000
3,0.075700,0.027398,26.845200,21.637700,26.847700,26.839000,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=2100, training_loss=0.26592905725751603, metrics={'train_runtime': 1017.8878, 'train_samples_per_second': 33.01, 'train_steps_per_second': 2.063, 'total_flos': 3635280787537920.0, 'train_loss': 0.26592905725751603, 'epoch': 3.0})

In [27]:
#Push model checkpoint to Hugging Face
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1717841133.6059b7c40b98.351.0:   0%|          | 0.00/8.61k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AnHoangbg/t5-small-finetuned-manimml-1.1/commit/d308de0589d08ca1af7dd6173caf667fbc85105c', commit_message='End of training', commit_description='', oid='d308de0589d08ca1af7dd6173caf667fbc85105c', pr_url=None, pr_revision=None, pr_num=None)

In [28]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("AnHoangbg/t5-small-finetuned-manimml-1.1")
model = AutoModelForSeq2SeqLM.from_pretrained("AnHoangbg/t5-small-finetuned-manimml-1.1")

tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

# Inference

In [29]:
text = "Visualize a neural network with the following architecture: It starts with a Convolutional2DLayer consisting of 1 feature map, each with a size of 7, using 3 filters with a spacing of 0.32 units. Next, there's another Convolutional2DLayer with 3 feature maps, each with a size of 5. It also employs 3 filters with a spacing of 0.32 units. Following that, there's a third Convolutional2DLayer with 5 feature maps, each with a size of 3. It uses 3 filters with a spacing of 0.18 units. After the convolutional layers, there are two FeedForwardLayers, each consisting of 3 nodes. The spacing between each layer is set at 0.25 units."
inputs = tokenizer(text, return_tensors='pt')

In [30]:
output = tokenizer.batch_decode(model.generate(inputs.input_ids, max_new_tokens=1024))

In [31]:
output

['<pad> nn=NeuralNetwork(Convolutional2DLayer(num_feature_maps=1, feature_map_size=7, filter_size=3, filter_spacing=0.32), Convolutional2DLayer(num_feature_maps=3, feature_map_size=5, filter_size=3, filter_spacing=0.32), Convolutional2DLayer(num_feature_maps=5, feature_map_size=3, filter_size=3, filter_spacing=0.18), FeedForwardLayer(num_nodes=3), FeedForwardLayer(num_nodes=3), FeedForwardLayer(num_nodes=3), FeedForwardLayer(num_nodes=3), layer_spacing: 0.25)</s>']